In [1]:
!lsb_release -a

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.3 LTS
Release:	22.04
Codename:	jammy


In [6]:
!sudo apt update && sudo apt upgrade -y
!sudo apt install nvidia-driver-550 nvidia-dkms-550

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,157 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [2,958 kB]
Hit:14 https://ppa.lau

In [7]:
!nvidia-smi

Mon Aug 12 16:58:19 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [8]:
!echo "Setup CuQuantum..."
!wget -q https://developer.download.nvidia.com/compute/cuquantum/redist/cuquantum/linux-x86_64/cuquantum-linux-x86_64-24.08.0.5_cuda12-archive.tar.xz
!tar -xvf cuquantum-linux-x86_64-24.08.0.5_cuda12-archive.tar.xz
!rm -rf cuquantum-linux-x86_64-24.08.0.5_cuda12-archive.tar.xz

!echo "Addinv Env variables..."

%env CUQUANTUM_ROOT=/content/cuquantum-linux-x86_64-24.08.0.5_cuda12-archive
%env LD_LIBRARY_PATH=/content/cuquantum-linux-x86_64-24.08.0.5_cuda12-archive/lib

Setup CuQuantum...
cuquantum-linux-x86_64-24.08.0.5_cuda12-archive/
cuquantum-linux-x86_64-24.08.0.5_cuda12-archive/distributed_interfaces/
cuquantum-linux-x86_64-24.08.0.5_cuda12-archive/distributed_interfaces/activate_mpi.sh
cuquantum-linux-x86_64-24.08.0.5_cuda12-archive/distributed_interfaces/cutensornet_distributed_interface_mpi.c
cuquantum-linux-x86_64-24.08.0.5_cuda12-archive/lib/
cuquantum-linux-x86_64-24.08.0.5_cuda12-archive/lib/libcutensornet.so.2
cuquantum-linux-x86_64-24.08.0.5_cuda12-archive/lib/libcutensornet.so
cuquantum-linux-x86_64-24.08.0.5_cuda12-archive/lib/libcutensornet.so.2.5.0.5
cuquantum-linux-x86_64-24.08.0.5_cuda12-archive/lib/libcutensornet_static.a
cuquantum-linux-x86_64-24.08.0.5_cuda12-archive/lib/libcustatevec.so.1.6.0.5
cuquantum-linux-x86_64-24.08.0.5_cuda12-archive/lib/libcustatevec.so
cuquantum-linux-x86_64-24.08.0.5_cuda12-archive/lib/libcustatevec.so.1
cuquantum-linux-x86_64-24.08.0.5_cuda12-archive/lib/libcustatevec_static.a
cuquantum-linux-x86_6

In [23]:
%%file h.h
#pragma once

#include <cuComplex.h>
#include <cmath>

const double half_superposition = 1.0/std::sqrt(2.0);

cuDoubleComplex H[] = {
    {half_superposition, 0.0}, {half_superposition, 0.0},
    {half_superposition, 0.0}, {-half_superposition, 0.0}
};

Overwriting h.h


In [31]:
%%file x.h
#pragma once

#include <cuComplex.h>

cuDoubleComplex X[] = {
    {0.0, 0.0}, {1.0, 0.0},
    {1.0, 0.0}, {0.0, 0.0},
};


Writing x.h


In [32]:
%%file bell.cu
#include <cuda_runtime_api.h>
#include <cuComplex.h>
#include <custatevec.h>
#include <iostream>
#include <cmath>

#include "./x.h"
#include "./h.h"

using std::cout;
using std::endl;

int main(void)
{
  cudaSetDevice(0);

  int n_devices;
  cudaError_t cuda_error = cudaGetDeviceCount(&n_devices);

  if (cuda_error != cudaSuccess)
  {
    cout << "cudaGetDeviceCount Error: "
         << cudaGetErrorString(cuda_error) << endl;
  }
  else
  {
    cout << "Total CUDA Devices: " << n_devices << endl;
  }

  custatevecHandle_t handle;
  custatevecCreate(&handle);

  const int n_qubits = 2;
  const int state_size = std::pow(2, n_qubits);

  cuDoubleComplex *state = new cuDoubleComplex[state_size];

  cuDoubleComplex *statevector;
  cudaMalloc((void **)&statevector, state_size * sizeof(cuDoubleComplex));

  cudaMemcpy(
      statevector,
      state,
      state_size * sizeof(cuDoubleComplex),
      cudaMemcpyHostToDevice);


  custatevecInitializeStateVector(handle, statevector, CUDA_C_64F, n_qubits, CUSTATEVEC_STATE_VECTOR_TYPE_ZERO);

  int hadamard_target[] = {0};
  custatevecApplyMatrix(
      handle,
      statevector,
      CUDA_C_64F,
      1,//n_qubits
      H, //hadamard gate
      CUDA_C_64F,
      CUSTATEVEC_MATRIX_LAYOUT_COL,
      0, //no adjoint
      hadamard_target, //target
      1, //n_targets
      nullptr, // controls
      nullptr, // control bit-string
      0,       // n controls
      CUSTATEVEC_COMPUTE_64F,
      nullptr,
      0);

  int cnot_target[] = {1};
  int cnot_control[] = {0};
  custatevecApplyMatrix(
      handle,
      statevector,
      CUDA_C_64F,
      n_qubits,//n_qubits
      X, //cnot(x controlled) gate
      CUDA_C_64F,
      CUSTATEVEC_MATRIX_LAYOUT_COL,
      0, //no adjoint
      cnot_target, //target
      1, //n_targets
      cnot_control, // controls
      nullptr, // control bit-string
      1,       // n controls
      CUSTATEVEC_COMPUTE_64F,
      nullptr,
      0);

  cudaMemcpy(
      state,
      statevector,
      state_size * sizeof(cuDoubleComplex),
      cudaMemcpyDeviceToHost);

  for (int i = 0; i < state_size; i++)
  {
    cuDoubleComplex c = state[i];
    cout << (double)c.x << endl;
  }

  custatevecDestroy(handle);
  cudaFree(statevector);
  delete state;

  return 0;
}

Overwriting bell.cu


In [33]:
!nvcc bell.cu -I${CUQUANTUM_ROOT}/include -L${CUQUANTUM_ROOT}/lib -lcustatevec -o bell

In [34]:
!./bell

Total CUDA Devices: 1
0.707107
0
0
0.707107
